In [1]:
import datetime
import math

import numpy as np
import pandas as pd
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline


In [2]:

def bayes(train_x,train_y,data,trainWordCount,testing_data,unigram,vector,indexes2=None,):

    indexes = []
    for row in train_y.index:
        indexes.append(row)

    if(indexes2 == None):
        for a,value in enumerate(train_x.toarray()):
            category = data["Category"][indexes[a]]
            trainWordCount[category] = np.sum([trainWordCount[category],value],axis=0)
        trainwordCountCopy = trainWordCount.copy()
    else:
        for a,value in enumerate(train_x.toarray()):
            category = data["Category"][indexes[a]]
            temp = np.zeros(len(indexes2))
            for count,b in enumerate(indexes2):
                temp[count] = value[b]
            trainWordCount[category] = np.sum([trainWordCount[category],temp],axis=0)

        trainwordCountCopy = trainWordCount.copy()


    bow = {}
    if indexes2==None:
        for a,value in enumerate(trainwordCountCopy):
            bow[categories[a]] = {}
            temp = {}
            for b,val in enumerate(vector.get_feature_names_out()):
                temp.update({val:trainwordCountCopy[a][b]})
            bow[categories[a]] = temp
    else:
        for a,value in enumerate(trainwordCountCopy):
            bow[categories[a]] = {}
            temp = {}
            countx = 0
            for val in (indexes2):
                temp.update({indexes2[val]:trainwordCountCopy[a][countx]})
                countx += 1
            bow[categories[a]] = temp
    results = testing_data["Category"]

    (true,false) = 0,0
    if(indexes2 == None):
        constx = len(vector.get_feature_names_out())
    else:
        constx = len(indexes2)

    time = datetime.datetime.now()
    for count,a in enumerate(testing_data["Text"]):
        mx = -math.inf
        maxindex = 0
        for b in bow:
            extend = 0
            if len(bow) == 0:
                print("bow:",bow)
            cons = math.log2(len(bow[b]) / len(train_y))
            tr = np.sum(trainwordCountCopy[cat[b]])
            if(unigram==False):
                for c in range(len(a.split(" "))-1):
                    word = str(a.split(" ")[c])  + " " + str(a.split(" ")[c+1])
                    try:
                        if bow[b][word] == 0:
                            extend = constx
                            break
                    except Exception:
                        extend = constx
                        break

                for c in range(len(a.split(" "))-1):
                    word = str(a.split(" ")[c])  + " " + str(a.split(" ")[c+1])
                    try:
                        number = bow[b][word]
                        if(extend != 0):
                            if bow[b][word] != 0:
                                cons += math.log2((number +1) / (tr + extend))
                            else:
                                cons += math.log2((1) / (tr + extend))

                        else:
                            cons += math.log2((number) / (tr))
                    except Exception:
                        cons += math.log2((1) / (tr + extend))
            else:
                for c in a.split(" "):
                    try:
                        if bow[b][c] == 0:
                            extend = constx
                            break
                    except Exception:
                        extend = constx
                        break
                for c in a.split(" "):
                    try:
                        if c not in my_stop_words:
                            if(extend != 0):
                                if(bow[b][c] != 0):
                                    cons += math.log2((bow[b][c] +1) / (tr + extend))
                                else:
                                    cons += math.log2((1) / (tr+extend))
                            else:
                                cons += math.log2((number) / (tr))
                    except Exception:
                        cons += math.log2((1)/(tr+extend))




            if(cons > mx):
                mx = cons
                maxindex = cat[b]
        if(categories[maxindex] == categories[results[results.index[count]]]):
            true += 1
        else:
            false += 1


    
    return true/(true+false)
    

def tfidf(vector):
    specwords = []
    df4=pd.DataFrame()
    for a in range(5):
        arr = []
        corpus = training_data[training_data["Category"] ==a]["Text"]
        Z = vector.fit_transform(corpus)
        vocabulary = vector.get_feature_names_out()
        corpus = corpus.tolist()
        pipe = Pipeline([('count', CountVectorizer(vocabulary=vocabulary)),('tfid', TfidfTransformer())]).fit(corpus)
        idx = np.argpartition(pipe['tfid'].idf_, 10)
        ls = pipe['tfid'].idf_
        ls2 = pipe['tfid'].idf_
        for b in range(10):
            ind = np.argmin(ls)
            value = ls[ind]
            ls[ind] = math.inf
            temp = [vocabulary[ind],value]
            specwords.append(vocabulary[ind])
            arr.append(temp)
        df = pd.DataFrame(data = arr,
                      columns = ["Word","TFID"])
        arr = []
        
        for b in range(10):
            ind = np.argmax(ls2)
            value = ls2[ind]
            ls2[ind] = -math.inf
            temp = [vocabulary[ind],value]
            arr.append(temp)
        df2 = pd.DataFrame(data = arr,
                      columns = ["Word","TFID"])

        

        res = []
        for  m in specwords:
            if m not in res:
                res.append(m)
    
        df3 = pd.concat([df,df2],axis=1)
        df3.columns=pd.MultiIndex.from_product([[categories[a].upper()],["Effect of Presence","Effect of Absence"],["Word","TF-IDF"]],names=["Category","Effect"," "])
       
        df4 = pd.concat([df4,df3],axis=1)
    
    return res,df4

In [3]:
data = pd.read_csv("English Dataset.csv")
dataFrame = data.drop(["ArticleId"], axis=1)
categories = {0:"sport",1:"business",2:"politics",3:"entertainment",4:"tech"}
cat = {"sport":0,"business":1,"politics":2,"entertainment":3,"tech":4}
dataFrame["Category"] = dataFrame["Category"].map(cat).astype(int)
my_stop_words = text.ENGLISH_STOP_WORDS

data = dataFrame.iloc[:, :]
vectorizer = CountVectorizer(stop_words="english")
vectorizerwithStop = CountVectorizer()
vectorizer2 = CountVectorizer(analyzer='word', ngram_range=(2, 2))
vectorizer2withStop = CountVectorizer(stop_words="english",analyzer='word', ngram_range=(2, 2))
X = vectorizer.fit_transform(data["Text"])

wordCount = np.zeros((5,(X.toarray()).shape[1]))



for a,value in enumerate(X.toarray()):
    category = data["Category"][a]
    wordCount[category] = np.sum([wordCount[category],value],axis=0)

wordCountCopy = wordCount.copy()
wordnumberdf = pd.DataFrame()
for count,a in enumerate(wordCountCopy):
    dict = []
    for b in range(4):
        ind = np.argmax(a)
        word = vectorizer.get_feature_names_out()[ind]
        amount = int(a[ind])
        temp = [word,amount]
        dict.append(temp)
        a[ind] = 0
    
    tempdf = pd.DataFrame(data = dict,
                      columns = pd.MultiIndex.from_product([[categories[count].upper()],["Word","Word Count"]]))
    
    wordnumberdf=pd.concat([tempdf,wordnumberdf],axis=1)
    


In [4]:
training_data, testing_data = train_test_split(dataFrame, test_size=0.2, shuffle=True)

train_x = vectorizer.fit_transform(training_data["Text"])
train_y = training_data["Category"]
train_x_withStop = vectorizerwithStop.fit_transform(training_data["Text"])
trainWordCount = np.zeros((5,(train_x.toarray()).shape[1]))
trainWordCountwithStop = np.zeros((5,(train_x_withStop.toarray()).shape[1]))

a1=bayes(train_x,train_y,data,trainWordCount,testing_data,True,vectorizer)

b1=bayes(train_x_withStop,train_y,data,trainWordCountwithStop,testing_data,True,vectorizerwithStop)


train_x_bigram = vectorizer2.fit_transform(training_data["Text"])
train_y_bigram = training_data["Category"]
train_x_bigram_stop=vectorizer2withStop.fit_transform(training_data["Text"])
trainWordCount_bigram = np.zeros((5,(train_x_bigram.toarray()).shape[1]))
trainWordCount_bigram_stop=np.zeros((5,(train_x_bigram_stop.toarray()).shape[1]))

accdf=pd.DataFrame(0.0,index=range(2), columns=range(2))


c1=bayes(train_x_bigram,train_y_bigram,data,trainWordCount_bigram,testing_data,False,vectorizer2)
d1=bayes(train_x_bigram_stop,train_y_bigram,data,trainWordCount_bigram_stop,testing_data,False,vectorizerwithStop)



vectorizer3 = CountVectorizer()
newdata,tfidfWithStopword = tfidf(vectorizer3)
train_x_updated_1 = vectorizer3.fit_transform(training_data["Text"])
train_y_updated_1 = training_data["Category"]

vectorizer4 = CountVectorizer(stop_words="english")

newdata2,tfidfWithoutStopword = tfidf(vectorizer4)
train_x_updated_2 = vectorizer4.fit_transform(training_data["Text"])
train_y_updated_2 = training_data["Category"]

(indexes3,indexes4)  = ({},{})
for count,a in enumerate(vectorizer3.get_feature_names_out()):
    if a in newdata:
        indexes3[count] = a


for count,a in enumerate(vectorizer4.get_feature_names_out()):
    if a in newdata2:
        indexes4[count] = a

newCountArr = np.zeros((5,(len(indexes3))))
newCountArr2 = np.zeros((5,(len(indexes4))))
e1=bayes(train_x_updated_1,train_y_updated_1,data,newCountArr,testing_data,True,vectorizer3,indexes3)
f1=bayes(train_x_updated_2,train_y_updated_2,data,newCountArr2,testing_data,True,vectorizer4,indexes4)

vectorizer5 = CountVectorizer(analyzer='word', ngram_range=(2, 2))
vectorizer6withStop = CountVectorizer(stop_words="english",analyzer='word', ngram_range=(2, 2))

accdf[0][0]=a1
accdf[1][0]=b1
accdf[0][1]=c1
accdf[1][1]=d1

accdf.index =["Unigram","Bigram"]
accdf.columns=pd.MultiIndex.from_product([["Included","Not Included"]],names = ["Stop-word"])


# Report
## Part I

In [5]:
wordnumberdf

TECH            ENTERTAINMENT               POLITICS            BUSINESS  \
     Word Word Count          Word Word Count        Word Word Count     Word   
0    said       1064          said        594        said       1445     said   
1  people        647          film        583          mr       1073     year   
2      mr        349          best        430      labour        494       mr   
3     new        349          year        315  government        464   market   

                SPORT             
  Word Count     Word Word Count  
0       1100     said        636  
1        456     game        356  
2        393     year        331  
3        284  england        329

Guessing a category from the headline is possible for some categories but the accuracy of guessing by looking at a single word is low. For example, if we see "people", we can say it may be tech but, the most repeated word except the stop words in each category is "said". When we see "said" in headline , category may be anyone of categories. 

### Accuracy

In [6]:
accdf

Stop-word,Included,Not Included
Unigram,0.986577,0.983221
Bigram,0.976510,0.161074


## Part II
First of all, deleting stop words doesn't cause big changes. Because all of the categories
have lots of stop words. It can cause small changes. Secondly, unigram's accuracy and bigram's accuracy
are very close too. Both of them specify the categories well. Average accuracy is 0.95. The accuracy
depends on the training data/test data which was shuffled. 


### • Table of the most and the least influential words, including stop-words

In [7]:
tfidfWithStopword

Category              SPORT                                        \
Effect   Effect of Presence           Effect of Absence             
                       Word    TF-IDF              Word    TF-IDF   
0                       the  1.000000                00  5.934474   
1                        to  1.003604                01  5.934474   
2                        of  1.014493                04  5.934474   
3                       and  1.018149                05  5.934474   
4                        in  1.018149               05m  5.934474   
5                       for  1.078545               060  5.934474   
6                       but  1.134560               093  5.934474   
7                        at  1.142824            09secs  5.934474   
8                        on  1.159561             1000m  5.934474   
9                      with  1.193899               102  5.934474   

Category           BUSINESS                                        \
Effect   Effect of Presence           Effect of Absence             
                       Word    TF-IDF              Word    TF-IDF   
0                        of  1.000000             000bn  5.912655   
1                       the  1.000000                01  5.912655   
2                        to  1.000000                03  5.912655   
3                        in  1.003683                04  5.912655   
4                       and  1.007380               041  5.912655   
5                      said  1.088349              04bn  5.912655   
6                       for  1.096414               050  5.912655   
7                      that  1.121005                06  5.912655   
8                        is  1.125163               069  5.912655   
9                        it  1.133531                07  5.912655   

Category           POLITICS                                        \
Effect   Effect of Presence           Effect of Absence             
                       Word    TF-IDF              Word    TF-IDF   
0                        in  1.000000                00  5.691348   
1                       the  1.000000              0001  5.691348   
2                        to  1.000000                05  5.691348   
3                       and  1.004598            050505  5.691348   
4                        of  1.004598                06  5.691348   
5                       for  1.051776               072  5.691348   
6                        is  1.051776              0800  5.691348   
7                        on  1.056619                09  5.691348   
8                        be  1.066375             100bn  5.691348   
9                      said  1.071289              100m  5.691348   

Category      ENTERTAINMENT                                        \
Effect   Effect of Presence           Effect of Absence             
                       Word    TF-IDF              Word    TF-IDF   
0                       the  1.000000             001st  5.727388   
1                       and  1.004435               007  5.727388   
2                        in  1.004435              0400  5.727388   
3                        of  1.008889               056  5.727388   
4                        to  1.022372              0700  5.727388   
5                        on  1.073427              0800  5.727388   
6                       for  1.082997              0845  5.727388   
7                       was  1.157845              0900  5.727388   
8                       has  1.189426               102  5.727388   
9                      with  1.194788               103  5.727388   

Category               TECH                                        
Effect   Effect of Presence           Effect of Absence            
                       Word    TF-IDF              Word    TF-IDF  
0                       and  1.000000                00  5.620059  
1                       the  1.000000              0051  5.620059  
2                        to  1.000000               007  5.620059  


### • Table of the most and the least influential words, except stop-words

In [8]:
tfidfWithoutStopword

Category              SPORT                                        \
Effect   Effect of Presence           Effect of Absence             
                       Word    TF-IDF              Word    TF-IDF   
0                      said  1.271035                00  5.934474   
1                      year  1.616986                01  5.934474   
2                      time  1.678861                04  5.934474   
3                       win  1.714966                05  5.934474   
4                      game  1.823600               05m  5.934474   
5                     world  1.945490               060  5.934474   
6                      just  1.973661               093  5.934474   
7                      team  2.002648            09secs  5.934474   
8                       won  2.042654             1000m  5.934474   
9                       old  2.084326               102  5.934474   

Category           BUSINESS                                        \
Effect   Effect of Presence           Effect of Absence             
                       Word    TF-IDF              Word    TF-IDF   
0                      said  1.088349             000bn  5.912655   
1                      year  1.458308                01  5.912655   
2                       new  1.810012                03  5.912655   
3                    market  1.843628                04  5.912655   
4                   company  1.852212               041  5.912655   
5                      firm  1.852212              04bn  5.912655   
6                        mr  2.010682               050  5.912655   
7                     chief  2.041454                06  5.912655   
8                      2004  2.051925               069  5.912655   
9                     years  2.117166                07  5.912655   

Category           POLITICS                                        \
Effect   Effect of Presence           Effect of Absence             
                       Word    TF-IDF              Word    TF-IDF   
0                      said  1.071289                00  5.691348   
1                        mr  1.254596              0001  5.691348   
2                government  1.394062                05  5.691348   
3                    people  1.588705            050505  5.691348   
4                  minister  1.622321                06  5.691348   
5                    labour  1.665996               072  5.691348   
6                       new  1.730535              0800  5.691348   
7                  election  1.740104                09  5.691348   
8                     party  1.759522             100bn  5.691348   
9                      told  1.779325              100m  5.691348   

Category      ENTERTAINMENT                                        \
Effect   Effect of Presence           Effect of Absence             
                       Word    TF-IDF              Word    TF-IDF   
0                      said  1.284737             001st  5.727388   
1                      year  1.450722               007  5.727388   
2                      film  1.738404              0400  5.727388   
3                       new  1.795562               056  5.727388   
4                      best  1.966188              0700  5.727388   
5                      star  2.038508              0800  5.727388   
6                        uk  2.116470              0845  5.727388   
7                 including  2.130076              0900  5.727388   
8                      time  2.130076               102  5.727388   
9                     years  2.130076               103  5.727388   

Category               TECH                                        
Effect   Effect of Presence           Effect of Absence            
                       Word    TF-IDF              Word    TF-IDF  
0                      said  1.114709                00  5.620059  
1                    people  1.269781              0051  5.620059  
2                       new  1.461176               007  5.620059  


## Part III
In this part,we want to find 10 words whose presence most strongly predicts that the article belongs to specific category, and 10 words whose absence most strongly predicts that the article belongs to specific category. We need Term Frequency - Inverce Document Frequency to do that. After calculating tf-idf value for all the words, the presence of words have lowest tf-idf value  most strongly predicts the article belongs to specific category. Predictably, the absence of words have highest tf-idf value most strongly predicts the article belongs to specific category. When we do this without deleting stop words, we can see the accuracy is too low. Its average is 0.16-0.20 because almost all stop words exist in each category. This makes predicting too hard.It is like predicting randomly, and the accuracy is very close to 1/5. When we delete the stop words, accuracy increases. Because we get the most 10 important words of each category, but it is not enough too. The average of accuracy is 0.30-0.35 . It's reason is texts have a lot of words, limiting them with low size words mean that probability of predicting correct is low.

Accuracy of collapsed data 

In [9]:
e1

0.1610738255033557

Accuracy of collapsed data except stop-words

In [10]:
f1

0.31543624161073824